# reshape in function using scipy optimize 

In [ ]:
import numpy as np
import cvxpy as cvx
import scipy
from scipy import optimize
from scipy.optimize import minimize

# objective function to minimize:
# Sum (W - Sum a*s)

N = 64 #number of filters
M = 18

#filter sizes:

n = 5
m = 3

#generate random sequence of filters
W = []
for i in range(N):
    W.append(np.random.rand(n,m))


#definition of the objective function
def fun(params, W, n, m, N, M):

    #recovering the original structure of parameters
    A = params[:N*M].reshape((N,M))
    X = params[N*M:].reshape((M,n,m))

    #print("A.Shape: ", A.shape)
    #print("X.Shape: ", X.shape)
    
    res = 0
    for i in range(N):
        
        # calculating the linear combination of matrices
        a_X = 0
        for j in range(M):
            a_X += A[i,j]*X[j,:,:]
            #print("-"*5,j)
            
        # calculating sum of
        res += np.linalg.norm(W[i] - a_X, 2)
        
    return res


shape = (n,m)
params = []

#initializing coefficients for linear combination
params = np.random.rand(N*M+M*n*m)  # N*M for lc coefiicients and m*n*M for matrix weights

res = scipy.optimize.minimize(fun, params, args=(W,n,m,N,M), callback=None)

vals = res['x']

A = vals[:N*M].reshape((N,M))
X = vals[N*M:].reshape((M,n,m))

#print ("weights are:")
#print (A)

#print ("basis filters are:")
#print (W)

#random check
for i in range(N):
    coeff_i = A[i,:]
    A_sum = 0
    for j in range(M):
        A_sum += coeff_i[j]*X[j]
    print ("approximation of matrix", i)
    print (np.round((np.linalg.norm(W[i] - A_sum, 2))*100, 5), "%")

In [ ]:
X = params[1].reshape((M,n,m))
A = params[0].reshape((N,M))

print(A[0,:])
print(X[1,:,:])

In [ ]:
import numpy as np
import cvxpy as cvx
import scipy
from scipy import optimize
from scipy.optimize import minimize


# objective function to minimize:
# Sum (W - Sum a*s)

N = 64 #number of filters
M = 15

#filter sizes:
n = 5
m = 5

#generate random sequence of filters

W = np.random.rand(N, n*m) 

# starting point
A = np.random.rand(N, M)
X = np.random.rand(M, n*m)

Max_Iter = 300
A_var = cvx.Variable(N, M)
X_var = cvx.Variable(M, n*m)


for i in range(Max_Iter):
    
    if i%5 == 0:
        print(i)
    
    objective_A = cvx.Minimize(cvx.norm(W - A_var*X,1))
    problem_A = cvx.Problem(objective_A, [])
    result_A = problem_A.solve() #solver = "SCS"
    
    if (np.linalg.norm(A - A_var.value,2) < 1e-7):
        print ("early termination A",i)
        break
    
    A = A_var.value
    
    objective_X = cvx.Minimize(cvx.norm(W - A*X_var,1))
    problem_X = cvx.Problem(objective_X, [])
    result_X  = problem_X.solve()
    
    if (np.linalg.norm(X - X_var.value,2) < 1e-7):
        print ("early termination X",i)
        break
    
    X = X_var.value
    
    
print((np.linalg.norm(W - np.matmul(A,X),2)/(np.linalg.norm(W,2)))*100,"%")

#recovering the original structure
W = np.reshape(W,(N,n,m))

A = A.view(type = np.ndarray)
X = X.view(type = np.ndarray)

A = np.reshape(A,(N,M))
X = np.reshape(X,(M,n,m))

#check
for i in range(N):
    A_sum = 0
    for j in range(M):
        A_sum += A[i,j]*X[j,:,:]
    print ("approximation of matrix", i)
    print (np.round((np.linalg.norm(W[i,:,:] - A_sum,1))*100, 2), "%")

In [ ]:
print (np.matrix.round(X[2,:,:],3))

# vectorized implementation of basis decomposition with nuclear norm

In [ ]:
import numpy as np
import cvxpy as cvx
import scipy
from scipy import optimize
from scipy.optimize import minimize

# returns low-basis representation of original weights W with shape N x n x m by
# linear combination of N matrices n x m with coefficients stored in matrix A
def find_basis(W, M, lmbd, Max_Iter, norm,tol):
    
    N = W.shape[0]
    n = W.shape[1]
    m = W.shape[2]
    
    # starting point
    W = W.reshape(N,n*m)
    
    A = np.random.rand(N, M)
    X = np.random.rand(M, n*m)

    A_var = cvx.Variable(N, M)
    X_var = cvx.Variable(M, n*m)


    for i in range(Max_Iter):

        if i%1 == 0:
            print(i)

        objective_A = cvx.Minimize(cvx.norm(W - A_var*X,norm))
        problem_A = cvx.Problem(objective_A, [])
        result_A = problem_A.solve() 

        if (np.linalg.norm(A - A_var.value,norm) < tol):
            print ("early termination A",i)
            break

        A = A_var.value

        #adding nuclear norm
        sum_nucl = 0
        slvr = None
        if lmbd != 0:
            slvr = "SCS"
            for j in range(M):
                x = X_var[j,:]
                x = cvx.reshape(x,n,m)
                sum_nucl += cvx.norm(x, 'nuc')

        objective_X = cvx.Minimize(cvx.norm(W - A*X_var,norm) + lmbd*sum_nucl)
        problem_X = cvx.Problem(objective_X, [])
        result_X  = problem_X.solve(solver = slvr)

        if (np.linalg.norm(X - X_var.value,norm) < tol):
            print ("early termination X",i)
            break

        X = X_var.value


    #print((np.linalg.norm(W - np.matmul(A,X),2)/(np.linalg.norm(W,2)))*100,"%")

    #recovering the original structure
    A = A.view(type = np.ndarray)
    X = X.view(type = np.ndarray)

    A = np.reshape(A,(N,M))
    X = np.reshape(X,(M,n,m))
    
    return A, X


# N = 64 #number of filters


# #filter size:
# n = 5
# m = 5

# #generate random sequence of filters

# W = np.random.rand(N,n,m) 
M = 14
#loading data
data = np.load('/home/pavel/Documents/Skoltech/NLA_Project/variables/scheme1_fr.npz')
shp = data['conv1/weights'].shape
new_weights = np.zeros((shp[0],shp[1],shp[2],M))
coeffs = np.zeros(shp[2],shp[3],M)
data = data['conv1/weights']
data = np.swapaxes(data, 0,3 )
data = np.swapaxes(data, 1,2 )
data = np.swapaxes(data, 1,3 )

#now channels axis is the last one
for c in range(data.shape[3]):
    print ("c = ",c)
    W = data[:,:,:,c]
    A, X = find_basis(W, M, lmbd = 0, Max_Iter = 100, norm = 1, tol = 1e-7)
    print ("done")
    #check
    for i in range(data.shape[0]):
        A_sum = 0
        for j in range(M):
            A_sum += A[i,j]*X[j,:,:]
        print ("approximation of matrix", i)
        print (np.round((np.linalg.norm(W[i,:,:] - A_sum,1))*100, 2), "%")
        print ("and some random?")
        print (np.round((np.linalg.norm(W[i,:,:] - np.random.rand(n,m)))*100, 2), "%")

In [ ]:
np.set_printoptions(suppress=True)
U,Sigma, V = np.linalg.svd(X[5,:,:])
print(np.matrix.round(Sigma,3))


# decomposition using 1-rank matrices

In [ ]:
import numpy as np
import cvxpy as cvx
import scipy
from scipy import optimize
from scipy.optimize import minimize


def find_1_rank_decomposition(W, N,K, n, m, Max_Iter, norm,tol):
    # starting point
    W = W.reshape(N,n*m)
    # initializing variable to optimize
    v_var = []
    h_var = []
    v = []
    h = []
    for k in range(K):
        v_var.append(cvx.Variable(n,N))
        h_var.append(cvx.Variable(N,m))

        # starting point 
        v.append(np.random.rand(n,N))
        h.append(np.random.rand(N,m))
    
    for i in range(Max_Iter):

        if i%10 == 0:
            print(i)
            
        fun_v = 0
        for j in range(N):
            for k in range(K):
            
                #temp1 = v_var[:,j]*np.array([h[j,:]])
                if np.array([h[k][j,:]]).shape == (1,m):
                    if k == 0:
                        temp1 = v_var[k][:,j]*np.array([h[k][j,:]])
                    else: 
                        temp1 += v_var[k][:,j]*np.array([h[k][j,:]])
                else:
                    if k == 0:
                        temp1 = v_var[k][:,j]*np.array(h[k][j,:])
                    else:
                        temp1 += v_var[k][:,j]*np.array(h[k][j,:])
                        
            temp1 = cvx.reshape(temp1,n*m,1)
            fun_v += cvx.norm(W[j,:] - temp1,norm)
        
        
        objective_v = cvx.Minimize(fun_v)
        problem_v = cvx.Problem(objective_v, [])
        result_v = problem_v.solve() #solver = "SCS"
        
        if (np.linalg.norm(v[0] - v_var[0].value,norm) < tol):
            print ("early termination v",i)
            break

        for k in range(K): 
            v[k] = v_var[k].value
            
        fun_h = 0

        for j in range(N):
            for k in range(K):
                if k == 0:
                    temp2 = np.array(v[k][:,j])*h_var[k][j,:]
                else:
                    temp2 += np.array(v[k][:,j])*h_var[k][j,:]
            temp2 = cvx.reshape(temp2,n*m,1)
            fun_h += cvx.norm(W[j,:] - temp2,norm)
               
        objective_h = cvx.Minimize(fun_h)
        problem_h = cvx.Problem(objective_h, [])
        result_h  = problem_h.solve() #solver = "SCS"
        if (np.linalg.norm(h[0] - h_var[0].value,norm) < tol):
            print ("early termination X",i)
            break

        for k in range(K): 
            h[k] = h_var[k].value
    #recovering the original structure
    for k in range(K):
        h[k] = h[k].view(type = np.ndarray)
        v[k] = v[k].view(type = np.ndarray)
    
    return h, v


N = 5 #number of filters

#filter size:
n = 5
m = 5

#generate random sequence of filters

W = np.random.rand(N,n,m) 
K = 2

H, V = find_1_rank_decomposition(W, N,K, n, m, Max_Iter = 300, norm = 1, tol = 1e-8)
#check
for i in range(N):

    res = 0
    for k in range(K):
        res += np.matmul(np.array([H[k][i,:]]).T,np.array([V[k][:,i]]))
        
    print ("approximation of matrix", i, ",",
          np.round((np.linalg.norm(W[i,:,:] - res,1))*100, 1), "%")
    print ("and comparison with random matrix?",i,",",
            np.round((np.linalg.norm(W[i,:,:] - np.random.rand(n,m)))*100, 1), "%")
    S,Z,D = np.linalg.svd(W[i,:,:])
    l = 1
    W_ = np.dot(S[:,:l], np.dot(np.diag(Z)[:l,:l], D[:l,:]))
    print ("SVD decomposition?",np.round(np.linalg.norm(W[i,:,:] - W_)*100, 1), "%")

In [ ]:
import numpy as np
import cvxpy as cvx
import scipy
from scipy import optimize
from scipy.optimize import minimize


# objective function to minimize:
# Sum (W - Sum a*s)

N = 15 #number of filters
M = 8

#filter sizes:
n = 3
m = 3

#generate random sequence of filters

W = np.random.rand(N, n, m) 

# starting point
A = np.random.rand(N,M)
X = np.random.rand(M,n,m)

S = []
for i in range(M):
    S.append(cvx.Variable(n,m))

# S = cvx.Variable(M,n,m)
A_var = cvx.Variable(N, M)

Max_Iter = 5


for i in range(Max_Iter):
    
    if i%5 == 0:
        print(i)
    
    fun_A = 0
    f_A = cvx.Variable(n,m)
    for p in range(N): 
        f_A += W[p,:,:]
        for k in range(M):
            f_A -= A_var[p,k]*X[k,:,:]
        fun_A += cvx.norm(f_A,1)

    objective_A = cvx.Minimize(fun_A)
    problem_A = cvx.Problem(objective_A, [])
    result_A = problem_A.solve(solver = "SCS") #solver = "SCS"
    if (np.linalg.norm(A - A_var.value,2) < 1e-7):
        print ("early termination A",i)
        break
    
    A = A_var.value
    
    fun_X = 0
    f_X = cvx.Variable(n,m)
    for p in range(N):
        f_X += W[p,:,:]  
        for k in range(M):
            f_X -= A[p,k]*S[k]
        fun_X += cvx.norm(f_X,1)
        #fun_X += 1.5*cvx.norm(S[k],'nuc')
    
    objective_X = cvx.Minimize(fun_X)
    problem_X = cvx.Problem(objective_X, [])
    result_X  = problem_X.solve(solver = "SCS")
    
    # here check for norm of X
    diff = 0
    for j in range(M):
        diff += np.linalg.norm(X[j,:,:] - S[j].value,2)
        
    if (diff < 1e-7):
        print ("early termination X",i)
        break
    
    for j in range(M):
        X[j,:,:] = S[j].value
    


#recovering the original structure

A = A.view(type = np.ndarray)
X = X.view(type = np.ndarray)

#check
for i in range(N):
    A_sum = 0
    for j in range(M):
        A_sum += A[i,j]*X[j,:,:]
    print ("approximation of matrix", i)
    print (np.round((np.linalg.norm(W[i,:,:] - A_sum,1))*100, 2), "%")

print(X[1,:,:])

In [2]:
U,Sigma, V = np.linalg.svd(X[3,:,:])
print(Sigma)

NameError: name 'np' is not defined

In [1]:
counter = 0

In [1]:
import numpy as np
import cvxpy as cvx
import scipy
from scipy import optimize
from scipy.optimize import minimize

# returns low-basis representation of original weights W with shape N x n x m by
# linear combination of N matrices n x m with coefficients stored in matrix A
def find_basis(W, M, lmbd_nuc, Max_Iter, norm, tol, lmbd_l1):
    
    N = W.shape[0]
    n = W.shape[1]
    m = W.shape[2]
    
    # starting point
    W = W.reshape(N,n*m)
    
    A = np.random.rand(N, M)
    X = np.random.rand(M, n*m)

    A_var = cvx.Variable(N, M)
    X_var = cvx.Variable(M, n*m)


    for i in range(Max_Iter):

        if i%1 == 0:
            #print(i)
            pass

        objective_A = cvx.Minimize(cvx.norm(W - A_var*X,norm))
        problem_A = cvx.Problem(objective_A, [])
        result_A = problem_A.solve() 

        if (np.linalg.norm(A - A_var.value,norm) < tol):
            print ("early termination A",i)
            break

        A = A_var.value

        #adding nuclear norm
        sum_nucl = 0
        slvr = None
        if lmbd_nuc != 0:
            slvr = "SCS"
            for j in range(M):
                x = X_var[j,:]
                x = cvx.reshape(x,n,m)
                sum_nucl += cvx.norm(x, 'nuc')
            sum_regul_norm = sum_nucl
            lmbd = lmbd_nuc
            
        #adding nuclear norm
        sum_l1 = 0
        if lmbd_l1 != 0:
            for j in range(M):
                x = X_var[j,:]
                x = cvx.reshape(x,n,m)
                sum_l1 += cvx.norm(x, 1)
            
            sum_regul_norm = sum_l1
            lmbd = lmbd_l1

        objective_X = cvx.Minimize(lmbd_l1*cvx.norm(W - A*X_var,norm) + lmbd*sum_regul_norm)
        problem_X = cvx.Problem(objective_X, [])
        result_X  = problem_X.solve(solver = slvr)

        if (np.linalg.norm(X - X_var.value,norm) < tol):
            print ("early termination X",i)
            break

        X = X_var.value


    #print((np.linalg.norm(W - np.matmul(A,X),2)/(np.linalg.norm(W,2)))*100,"%")

    #recovering the original structure
    A = A.view(type = np.ndarray)
    X = X.view(type = np.ndarray)

    A = np.reshape(A,(N,M))
    X = np.reshape(X,(M,n,m))
    
    return A, X


# N = 64 #number of filters


# #filter size:
# n = 5
# m = 5

# #generate random sequence of filters

# W = np.random.rand(N,n,m) 
M = 14
#loading data
data = np.load('variables/scheme1_fr.npz')
shp = data['conv1/weights'].shape
# new_weights = np.zeros((shp[0],shp[1],shp[2],M))
# coeffs = np.zeros(shp[2],shp[3],M)
data = data['conv1/weights']
C = data.shape[2]
N = data.shape[3]
data = data.reshape(shp[0],shp[1],-1)
data = np.swapaxes(data, 0,2 )
data = np.swapaxes(data, 1,2 )
W = data
#now channels axis is the last one

#A, X = find_basis(W, M, lmbd_nuc = 0, Max_Iter = 300, norm = 1, tol = 1e-7, lmbd_l1 = 1)

In [2]:
#check
def check(A, X):
    errors = []
    for i in range(C*N):
        A_sum = 0
        for j in range(M):
            A_sum += A[i,j]*X[j,:,:]
        errors.append((np.round((np.linalg.norm(W[i,:,:] - A_sum,1))*100, 1)))
        
    return [np.median(errors), np.mean(errors), np.max(errors)]

In [8]:
results = []
for lmbd in range(1, 15):
    A, X = find_basis(W, M, lmbd_nuc = 0, Max_Iter = 100, norm = 1, tol = 1e-7, lmbd_l1 = lmbd)
    results.append(check(A, X))
    print('#{}: Median: {}. Mean: {}. Max: {}'.format(lmbd, *results[-1]))

early termination X 44
#1: Median: 17.6. Mean: 16.559375000000003. Max: 28.5
early termination X 17
#2: Median: 17.65. Mean: 19.14375. Max: 36.3
early termination X 78
#3: Median: 15.6. Mean: 17.465625. Max: 36.2
early termination X 16
#4: Median: 16.85. Mean: 17.30625. Max: 33.6
early termination X 22
#5: Median: 16.700000000000003. Mean: 17.081249999999997. Max: 41.9
early termination X 27
#6: Median: 18.65. Mean: 18.65625. Max: 31.1
early termination X 71
#7: Median: 17.1. Mean: 16.89375. Max: 29.5
early termination X 17
#8: Median: 15.45. Mean: 17.64375. Max: 39.0
early termination X 61
#9: Median: 14.8. Mean: 16.56875. Max: 33.7
early termination X 19
#10: Median: 18.7. Mean: 18.184375000000003. Max: 32.3
early termination X 24
#11: Median: 16.0. Mean: 17.31875. Max: 32.1
early termination X 26
#12: Median: 16.049999999999997. Mean: 18.046875. Max: 36.2
early termination X 37
#13: Median: 18.5. Mean: 18.828125. Max: 31.3
early termination X 31
#14: Median: 15.75. Mean: 17.246875. 

In [ ]:
#save
A = A.reshape(C,N,M)
# A - matrix of coefficients N*C by M, X
np.savez('variables/scheme1_layer_2_fr_decomposition{}.npz'.format(counter), coeffs = A, basis = X )
counter += 1

In [ ]:
G = np.array([range(27)]).reshape(3,3,3)
print(G)

print(G.reshape(3,-1))